In [ ]:
# Here I make fresh data files for J. A large one to do experiments and several small ones,
# each a subset of the first one (run14jun21no14.txt) that are small enough
# to upload oto GitHub.

In [1]:
D.<x> = PolynomialRing(QQ)
R.<x> = PolynomialRing(QQ)
S = PowerSeriesRing(QQ,'x', default_prec=300)
L=LaurentSeriesRing(QQ,'x', default_prec=300)
#S.<x,y>=PowerSeriesRing(QQ,default_prec=300)
# Using "R = PolynomialRing(QQ,'x')" produces an error.
# Yet using S as defined here is fine (see definition of Fraleigh4 below)
# But S = PowerSeriesRing(QQ,'x','y') also produces an error.
def pochhammer(x,n):
    return product ([x+k for k in [0..n-1]])
def cRaleigh2(alpha,beta,nu):
    pchhmrA = pochhammer(alpha,nu)
    pchhmrB = pochhammer(beta,nu)
    fnu=factorial(nu)
    answer = D(pchhmrA*pchhmrB/fnu^2)
    return answer
def term(alpha,beta,p):
    return D(1/(alpha + p) + 1/(beta + p) - 2/(1 + p))
def eRaleigh(alpha, beta, nu):
    return D(sum([term(alpha, beta, p) for p in [0..nu - 1]]))
def FstarRaleigh2(alpha,beta,u,terms):
    fsr = sum([cRaleigh2(alpha, beta, nu)*eRaleigh(alpha,beta,nu)*u^nu for nu in [1..terms]])
    return D(fsr)
def Fraleigh2(alpha,beta,u,terms): 
    return D(sum([cRaleigh2(alpha,beta,nu)*u^nu for nu in [0..terms]]))
def FstarRaleigh3(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fsr2 = FstarRaleigh2(alpha,beta,x,n)
    return D(fsr2)
def Fraleigh4(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fr2 = Fraleigh2(alpha,beta,x,n)
    return D(fr2)
def exNo3c(n,m):
    a1 = S(x)*exp(S(FstarRaleigh3(n,m,x)/Fraleigh4(n,m,x))).O(2*n)
    # S(x) instead of x in df of a1 to avoid an error message
    # after use of the stripQuotationMarks command below.
    a2 = S(a1).O(n+1)
    return a2 # applying S to argument of exp is key.
def J(n,m):
    exn = exNo3c(n+2,m)
    jnm = 1/exn.reverse()
    return jnm.O(n+1) # Jtest is faster than J, and they agree.
def normalJ(n,m):
    f = J(n,m)
    return f.truncate(n+1)
def Jcoefficient(n,m):
    f = J(n+2,m)
    f1 = f.truncate(n+1)
    f1=f1.list()
    f1=Set(f1)
    g1 = f.truncate(n)
    g1 = g1.list()
    g1 = set(g1)
    h = f1.difference(g1)
    h = h.list()
    return h[0]
def xjNew(n,m):
    def xjNewStep(n,m):
        f=J(n,m)
        fl=f.list()
        rt=0
        for i in [0..len(fl)-1]:
            rt=rt+fl[i]*x**i*(m**3*2**6)**i
        return rt
    return xjNewStep(n-1,m)
def j(n,m):return L(xjNew(n,m)/x)
def polynomialCoefficient(n,poly):
    return poly.list()[n]
def leadingCoefficient(polynomial):
    dg=polynomial.degree()
    return polynomialCoefficient(dg,polynomial)
def factorpairToFactor(factorpair):
    (monomial,exponent)=factorpair
    return monomial^exponent
def numericalfactor(polynomial):
    fp=polynomial.factor()
    lst=[factorpairToFactor(factorpair) for factorpair in fp]
    return polynomial/product(lst)
def firstfactor(polynomial):
    fp=polynomial.factor()
    fp=FiniteEnumeratedSet(fp)
    return factorpairToFactor(fp.first())
def lastfactor(polynomial):
    fp=polynomial.factor()
    fp=FiniteEnumeratedSet(fp)
    return factorpairToFactor(fp.last())
def reduced(polynomial):
    return numericalfactor(polynomial)*lastfactor(polynomial)
def stripQuotationMarks(string):
    var('x')
    return sage_eval(string,locals={'x':x})
print 89

89


In [2]:
print(j(5,3));print("");print(J(5,3));print normalJ(5,3)

x^-1 + 744 + 196884*x + 21493760*x^2 + 864299970*x^3 + 20245856256*x^4

x^-1 + 31/72 + 1823/27648*x + 10495/2519424*x^2 + 1778395/18345885696*x^3 + 45767/34828517376*x^4 + 41650330075/3327916660110655488*x^5 + O(x^6)
x^-1 + 31/72 + 1823/27648*x + 10495/2519424*x^2 + 1778395/18345885696*x^3 + 45767/34828517376*x^4 + 41650330075/3327916660110655488*x^5


In [ ]:
## printed output has been snipped

import pickle
wfile=open('/Users/barrybrent/run1jun21no1.txt','a')
data=[]
for m in [3..500]:
    print m
    data=data+[[m,normalJ(200,m)]]
s=pickle.dumps(str(data))
wfile.write(s)
wfile.close()

In [12]:
import pickle
rfile = open('/Users/barrybrent/run1jun21no1.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)

polydata=[]
import time
wfile = open('/Users/barrybrent/run14jun21no1.txt','a') # xJ polynomials
for n in [0..5]:
    start = time.time()
    data=[]
    for k in [0..len(s)-1]:
        print (n,k,time.time()-start)
        m = s[k][0]
        poly=x*s[k][1] 
        cf=polynomialCoefficient(n,poly)
        data=data+[[m,cf*m^(2*n)]]
    rl=R.lagrange_polynomial(data)
    polydata=polydata+[[n,rl]] 
    finish = time.time()
    
t = pickle.dumps(str(polydata))
wfile.write(t)
wfile.close()

(0, 0, 0.0005190372467041016)
(0, 1, 0.5867929458618164)
(0, 2, 0.5968310832977295)
(0, 3, 0.6125519275665283)
(0, 4, 0.6232888698577881)
(0, 5, 0.6394929885864258)
(0, 6, 0.6516580581665039)
(0, 7, 0.6680080890655518)
(0, 8, 0.681251049041748)
(0, 9, 0.6977438926696777)
(0, 10, 0.7120490074157715)
(0, 11, 0.7290229797363281)
(0, 12, 0.7430300712585449)
(0, 13, 0.7619850635528564)
(0, 14, 0.7744600772857666)
(0, 15, 0.7921199798583984)
(0, 16, 0.8063499927520752)
(0, 17, 0.8236920833587646)
(0, 18, 0.839015007019043)
(0, 19, 0.8571569919586182)
(0, 20, 0.8711600303649902)
(0, 21, 0.8884000778198242)
(0, 22, 0.9030070304870605)
(0, 23, 0.9209709167480469)
(0, 24, 0.9359569549560547)
(0, 25, 0.9546129703521729)
(0, 26, 0.972667932510376)
(0, 27, 0.9911489486694336)
(0, 28, 1.0076770782470703)
(0, 29, 1.0260958671569824)
(0, 30, 1.03898286819458)
(0, 31, 1.0606060028076172)
(0, 32, 1.0764410495758057)
(0, 33, 1.0967988967895508)
(0, 34, 1.1136069297790527)
(0, 35, 1.1321370601654053)
(0, 

(1, 90, 1.6415209770202637)
(1, 91, 1.6643810272216797)
(1, 92, 1.681980848312378)
(1, 93, 1.7039690017700195)
(1, 94, 1.719547986984253)
(1, 95, 1.7404718399047852)
(1, 96, 1.758862018585205)
(1, 97, 1.7811667919158936)
(1, 98, 1.80035400390625)
(1, 99, 1.821767807006836)
(1, 100, 1.8412139415740967)
(1, 101, 1.8628020286560059)
(1, 102, 1.8805608749389648)
(1, 103, 1.9045119285583496)
(1, 104, 1.922353982925415)
(1, 105, 1.9442288875579834)
(1, 106, 1.9635698795318604)
(1, 107, 1.9849989414215088)
(1, 108, 2.003887891769409)
(1, 109, 2.0261240005493164)
(1, 110, 2.0428659915924072)
(1, 111, 2.0640528202056885)
(1, 112, 2.0836949348449707)
(1, 113, 2.1070728302001953)
(1, 114, 2.12680983543396)
(1, 115, 2.1492209434509277)
(1, 116, 2.1681528091430664)
(1, 117, 2.191340923309326)
(1, 118, 2.211488962173462)
(1, 119, 2.233670949935913)
(1, 120, 2.2517898082733154)
(1, 121, 2.2739758491516113)
(1, 122, 2.2929248809814453)
(1, 123, 2.3152029514312744)
(1, 124, 2.334758996963501)
(1, 125, 

(1, 381, 8.02663779258728)
(1, 382, 8.042930841445923)
(1, 383, 8.06964898109436)
(1, 384, 8.090204954147339)
(1, 385, 8.116183996200562)
(1, 386, 8.13820195198059)
(1, 387, 8.162336826324463)
(1, 388, 8.185050964355469)
(1, 389, 8.209969997406006)
(1, 390, 8.230479001998901)
(1, 391, 8.256110906600952)
(1, 392, 8.27593994140625)
(1, 393, 8.300745964050293)
(1, 394, 8.32244086265564)
(1, 395, 8.345923900604248)
(1, 396, 8.365358829498291)
(1, 397, 8.391170978546143)
(1, 398, 8.40976881980896)
(1, 399, 8.432955980300903)
(1, 400, 8.455151796340942)
(1, 401, 8.48006796836853)
(1, 402, 8.501574993133545)
(1, 403, 8.527076959609985)
(1, 404, 8.548027992248535)
(1, 405, 8.572367906570435)
(1, 406, 8.593324899673462)
(1, 407, 8.616694927215576)
(1, 408, 8.637576818466187)
(1, 409, 8.662168025970459)
(1, 410, 8.68342399597168)
(1, 411, 8.708096027374268)
(1, 412, 8.729184865951538)
(1, 413, 8.753852844238281)
(1, 414, 8.77134895324707)
(1, 415, 8.798255920410156)
(1, 416, 8.819429874420166)
(

(2, 182, 3.4618170261383057)
(2, 183, 3.484652042388916)
(2, 184, 3.50394606590271)
(2, 185, 3.5262179374694824)
(2, 186, 3.5453310012817383)
(2, 187, 3.5685341358184814)
(2, 188, 3.5874850749969482)
(2, 189, 3.6092569828033447)
(2, 190, 3.625092029571533)
(2, 191, 3.6484761238098145)
(2, 192, 3.6668360233306885)
(2, 193, 3.6912760734558105)
(2, 194, 3.7109620571136475)
(2, 195, 3.7334001064300537)
(2, 196, 3.754563093185425)
(2, 197, 3.7785980701446533)
(2, 198, 3.7973639965057373)
(2, 199, 3.820646047592163)
(2, 200, 3.839066982269287)
(2, 201, 3.8637030124664307)
(2, 202, 3.8839480876922607)
(2, 203, 3.906867027282715)
(2, 204, 3.9255259037017822)
(2, 205, 3.9488489627838135)
(2, 206, 3.966646909713745)
(2, 207, 3.9894700050354004)
(2, 208, 4.010955095291138)
(2, 209, 4.034218072891235)
(2, 210, 4.05438494682312)
(2, 211, 4.07759690284729)
(2, 212, 4.096338987350464)
(2, 213, 4.120573997497559)
(2, 214, 4.140887022018433)
(2, 215, 4.164745092391968)
(2, 216, 4.183716058731079)
(2, 2

(2, 487, 10.34418797492981)
(2, 488, 10.365870952606201)
(2, 489, 10.389912128448486)
(2, 490, 10.412019968032837)
(2, 491, 10.43874192237854)
(2, 492, 10.461197137832642)
(2, 493, 10.488358974456787)
(2, 494, 10.507401943206787)
(2, 495, 10.532189130783081)
(2, 496, 10.553821086883545)
(2, 497, 10.578529119491577)
(3, 0, 0.0005419254302978516)
(3, 1, 0.012157917022705078)
(3, 2, 0.021958112716674805)
(3, 3, 0.03646993637084961)
(3, 4, 0.04679608345031738)
(3, 5, 0.06235814094543457)
(3, 6, 0.07402610778808594)
(3, 7, 0.0903630256652832)
(3, 8, 0.1038510799407959)
(3, 9, 0.12183308601379395)
(3, 10, 0.13615703582763672)
(3, 11, 0.15316009521484375)
(3, 12, 0.16680502891540527)
(3, 13, 0.18454909324645996)
(3, 14, 0.19695496559143066)
(3, 15, 0.21445608139038086)
(3, 16, 0.22887110710144043)
(3, 17, 0.24715304374694824)
(3, 18, 0.2627289295196533)
(3, 19, 0.28177309036254883)
(3, 20, 0.2962961196899414)
(3, 21, 0.31385207176208496)
(3, 22, 0.32871198654174805)
(3, 23, 0.3466160297393799

(3, 288, 5.787099123001099)
(3, 289, 5.811300039291382)
(3, 290, 5.831634044647217)
(3, 291, 5.855180978775024)
(3, 292, 5.876392126083374)
(3, 293, 5.900931119918823)
(3, 294, 5.920573949813843)
(3, 295, 5.944914102554321)
(3, 296, 5.964409112930298)
(3, 297, 5.989534139633179)
(3, 298, 6.012013912200928)
(3, 299, 6.03618597984314)
(3, 300, 6.055788040161133)
(3, 301, 6.080442905426025)
(3, 302, 6.0988991260528564)
(3, 303, 6.122927904129028)
(3, 304, 6.143567085266113)
(3, 305, 6.166513919830322)
(3, 306, 6.187452077865601)
(3, 307, 6.211184978485107)
(3, 308, 6.2326340675354)
(3, 309, 6.256968021392822)
(3, 310, 6.2785680294036865)
(3, 311, 6.301814079284668)
(3, 312, 6.322673082351685)
(3, 313, 6.348505020141602)
(3, 314, 6.369426012039185)
(3, 315, 6.392453908920288)
(3, 316, 6.4131739139556885)
(3, 317, 6.436368942260742)
(3, 318, 6.4526379108428955)
(3, 319, 6.47806191444397)
(3, 320, 6.498846054077148)
(3, 321, 6.523447036743164)
(3, 322, 6.5454161167144775)
(3, 323, 6.57120108

(4, 99, 1.7808208465576172)
(4, 100, 1.7996349334716797)
(4, 101, 1.8206689357757568)
(4, 102, 1.8381068706512451)
(4, 103, 1.8618049621582031)
(4, 104, 1.8789188861846924)
(4, 105, 1.900212049484253)
(4, 106, 1.9201140403747559)
(4, 107, 1.9427969455718994)
(4, 108, 1.9620230197906494)
(4, 109, 1.9863488674163818)
(4, 110, 2.0035810470581055)
(4, 111, 2.026033878326416)
(4, 112, 2.0454280376434326)
(4, 113, 2.0676398277282715)
(4, 114, 2.0868279933929443)
(4, 115, 2.109184980392456)
(4, 116, 2.127351999282837)
(4, 117, 2.1485278606414795)
(4, 118, 2.1669838428497314)
(4, 119, 2.187882900238037)
(4, 120, 2.205245018005371)
(4, 121, 2.2266368865966797)
(4, 122, 2.2448530197143555)
(4, 123, 2.268440008163452)
(4, 124, 2.2881569862365723)
(4, 125, 2.3091540336608887)
(4, 126, 2.3224689960479736)
(4, 127, 2.3461239337921143)
(4, 128, 2.3650059700012207)
(4, 129, 2.3857839107513428)
(4, 130, 2.4058799743652344)
(4, 131, 2.4278149604797363)
(4, 132, 2.4464809894561768)
(4, 133, 2.46907496452

(5, 189, 3.604804039001465)
(5, 190, 3.621109962463379)
(5, 191, 3.643193006515503)
(5, 192, 3.661425828933716)
(5, 193, 3.6861610412597656)
(5, 194, 3.7057628631591797)
(5, 195, 3.7277920246124268)
(5, 196, 3.74691104888916)
(5, 197, 3.770853042602539)
(5, 198, 3.7908740043640137)
(5, 199, 3.815027952194214)
(5, 200, 3.8340818881988525)
(5, 201, 3.857481002807617)
(5, 202, 3.8783018589019775)
(5, 203, 3.9019320011138916)
(5, 204, 3.9211490154266357)
(5, 205, 3.9448628425598145)
(5, 206, 3.963109016418457)
(5, 207, 3.9863169193267822)
(5, 208, 4.008205890655518)
(5, 209, 4.030321836471558)
(5, 210, 4.049770832061768)
(5, 211, 4.072922945022583)
(5, 212, 4.091331958770752)
(5, 213, 4.114428997039795)
(5, 214, 4.133565902709961)
(5, 215, 4.157220840454102)
(5, 216, 4.177963972091675)
(5, 217, 4.201397895812988)
(5, 218, 4.2221839427948)
(5, 219, 4.245009899139404)
(5, 220, 4.2646520137786865)
(5, 221, 4.286770820617676)
(5, 222, 4.303995847702026)
(5, 223, 4.327972888946533)
(5, 224, 4.3

(5, 486, 10.192790985107422)
(5, 487, 10.219805002212524)
(5, 488, 10.24146294593811)
(5, 489, 10.265496969223022)
(5, 490, 10.28760290145874)
(5, 491, 10.311774969100952)
(5, 492, 10.332551002502441)
(5, 493, 10.358611822128296)
(5, 494, 10.377184867858887)
(5, 495, 10.40300703048706)
(5, 496, 10.425189971923828)
(5, 497, 10.45064091682434)


In [4]:
import pickle
rfile = open('/Users/barrybrent/run14jun21no1.txt','r') # xJ polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)


for k in [0..3]:
    index=s[k][0]
    poly = s[k][1]
    print (index,poly)
    print "========================================================================="
    # these match Raleigh's eqn III as well.

(0, 1)
(1, 3/8*x^2 + 1/2)
(2, 69/1024*x^4 - 1/128*x^2 - 3/64)
(3, 1/128*x^6 - 29/864*x^4 + 1/216*x^2 + 1/54)


In [9]:
import pickle
rfile = open('/Users/barrybrent/run1jun21no1.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)

polydata=[]

wfile = open('/Users/barrybrent/run14jun21no6.txt','a') # xJ polynomials
for n in [0..5]:
   
    data=[]
    for k in [0..200]:
        print (n,k)
        m = s[k][0]
        poly=x*s[k][1] 
        cf=polynomialCoefficient(n,poly)
        data=data+[[m,cf*m^(2*n)]]
    rl=R.lagrange_polynomial(data)
    polydata=polydata+[[n-1,rl]] 

    
t = pickle.dumps(str(polydata))
wfile.write(t)
wfile.close()

(0, 0)
(0, 1)
(0, 2)
(0, 3)
(0, 4)
(0, 5)
(0, 6)
(0, 7)
(0, 8)
(0, 9)
(0, 10)
(0, 11)
(0, 12)
(0, 13)
(0, 14)
(0, 15)
(0, 16)
(0, 17)
(0, 18)
(0, 19)
(0, 20)
(0, 21)
(0, 22)
(0, 23)
(0, 24)
(0, 25)
(0, 26)
(0, 27)
(0, 28)
(0, 29)
(0, 30)
(0, 31)
(0, 32)
(0, 33)
(0, 34)
(0, 35)
(0, 36)
(0, 37)
(0, 38)
(0, 39)
(0, 40)
(0, 41)
(0, 42)
(0, 43)
(0, 44)
(0, 45)
(0, 46)
(0, 47)
(0, 48)
(0, 49)
(0, 50)
(0, 51)
(0, 52)
(0, 53)
(0, 54)
(0, 55)
(0, 56)
(0, 57)
(0, 58)
(0, 59)
(0, 60)
(0, 61)
(0, 62)
(0, 63)
(0, 64)
(0, 65)
(0, 66)
(0, 67)
(0, 68)
(0, 69)
(0, 70)
(0, 71)
(0, 72)
(0, 73)
(0, 74)
(0, 75)
(0, 76)
(0, 77)
(0, 78)
(0, 79)
(0, 80)
(0, 81)
(0, 82)
(0, 83)
(0, 84)
(0, 85)
(0, 86)
(0, 87)
(0, 88)
(0, 89)
(0, 90)
(0, 91)
(0, 92)
(0, 93)
(0, 94)
(0, 95)
(0, 96)
(0, 97)
(0, 98)
(0, 99)
(0, 100)
(0, 101)
(0, 102)
(0, 103)
(0, 104)
(0, 105)
(0, 106)
(0, 107)
(0, 108)
(0, 109)
(0, 110)
(0, 111)
(0, 112)
(0, 113)
(0, 114)
(0, 115)
(0, 116)
(0, 117)
(0, 118)
(0, 119)
(0, 120)
(0, 121)
(0, 122)
(0,

(4, 174)
(4, 175)
(4, 176)
(4, 177)
(4, 178)
(4, 179)
(4, 180)
(4, 181)
(4, 182)
(4, 183)
(4, 184)
(4, 185)
(4, 186)
(4, 187)
(4, 188)
(4, 189)
(4, 190)
(4, 191)
(4, 192)
(4, 193)
(4, 194)
(4, 195)
(4, 196)
(4, 197)
(4, 198)
(4, 199)
(4, 200)
(5, 0)
(5, 1)
(5, 2)
(5, 3)
(5, 4)
(5, 5)
(5, 6)
(5, 7)
(5, 8)
(5, 9)
(5, 10)
(5, 11)
(5, 12)
(5, 13)
(5, 14)
(5, 15)
(5, 16)
(5, 17)
(5, 18)
(5, 19)
(5, 20)
(5, 21)
(5, 22)
(5, 23)
(5, 24)
(5, 25)
(5, 26)
(5, 27)
(5, 28)
(5, 29)
(5, 30)
(5, 31)
(5, 32)
(5, 33)
(5, 34)
(5, 35)
(5, 36)
(5, 37)
(5, 38)
(5, 39)
(5, 40)
(5, 41)
(5, 42)
(5, 43)
(5, 44)
(5, 45)
(5, 46)
(5, 47)
(5, 48)
(5, 49)
(5, 50)
(5, 51)
(5, 52)
(5, 53)
(5, 54)
(5, 55)
(5, 56)
(5, 57)
(5, 58)
(5, 59)
(5, 60)
(5, 61)
(5, 62)
(5, 63)
(5, 64)
(5, 65)
(5, 66)
(5, 67)
(5, 68)
(5, 69)
(5, 70)
(5, 71)
(5, 72)
(5, 73)
(5, 74)
(5, 75)
(5, 76)
(5, 77)
(5, 78)
(5, 79)
(5, 80)
(5, 81)
(5, 82)
(5, 83)
(5, 84)
(5, 85)
(5, 86)
(5, 87)
(5, 88)
(5, 89)
(5, 90)
(5, 91)
(5, 92)
(5, 93)
(5, 94)
(5, 95)

In [10]:
import pickle
rfile = open('/Users/barrybrent/run14jun21no6.txt','r') # xJ polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)


for k in [0..3]:
    index=s[k][0]
    poly = s[k][1]
    print (index,poly)
    print "========================================================================="
    # these match Raleigh's eqn III as well.

(-1, 1)
(0, 3/8*x^2 + 1/2)
(1, 69/1024*x^4 - 1/128*x^2 - 3/64)
(2, 1/128*x^6 - 29/864*x^4 + 1/216*x^2 + 1/54)


In [20]:
import pickle
rfile = open('/Users/barrybrent/run1jun21no1.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
len(s)



498

In [27]:
import pickle
rfile = open('/Users/barrybrent/run1jun21no1.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)

polydata=[]

wfile = open('/Users/barrybrent/run14jun21no13.txt','a') # xJ polynomials
for n in [0..3]:
   
    data=[]
    for k in [0..2*n+1]:
        print (n,k)
        m = s[k][0]
        poly=x*s[k][1] 
        cf=polynomialCoefficient(n,poly)
        data=data+[[m,cf*m^(2*n)]]
    rl=R.lagrange_polynomial(data)
    polydata=polydata+[[n-1,rl]] 

    
t = pickle.dumps(str(polydata))
wfile.write(t)
wfile.close()

(0, 0)
(0, 1)
(1, 0)
(1, 1)
(1, 2)
(1, 3)
(2, 0)
(2, 1)
(2, 2)
(2, 3)
(2, 4)
(2, 5)
(3, 0)
(3, 1)
(3, 2)
(3, 3)
(3, 4)
(3, 5)
(3, 6)
(3, 7)


In [28]:
import pickle
rfile = open('/Users/barrybrent/run14jun21no13.txt','r') # xJ polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)


for k in [0..3]:
    index=s[k][0]
    poly = s[k][1]
    print (index,poly)
    print "========================================================================="

(-1, 1)
(0, 3/8*x^2 + 1/2)
(1, 69/1024*x^4 - 1/128*x^2 - 3/64)
(2, 1/128*x^6 - 29/864*x^4 + 1/216*x^2 + 1/54)


In [30]:
import pickle
rfile = open('/Users/barrybrent/run1jun21no1.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)

polydata=[]

wfile = open('/Users/barrybrent/run14jun21no13a.txt','a') # xJ polynomials
for n in [0..3]:
   
    data=[]
    for k in [0..2*n+2]: #increase the range to see whether the interpolations have stabilized
        print (n,k)
        m = s[k][0]
        poly=x*s[k][1] 
        cf=polynomialCoefficient(n,poly)
        data=data+[[m,cf*m^(2*n)]]
    rl=R.lagrange_polynomial(data)
    polydata=polydata+[[n-1,rl]] 

    
t = pickle.dumps(str(polydata))
wfile.write(t)
wfile.close()

(0, 0)
(0, 1)
(0, 2)
(1, 0)
(1, 1)
(1, 2)
(1, 3)
(1, 4)
(2, 0)
(2, 1)
(2, 2)
(2, 3)
(2, 4)
(2, 5)
(2, 6)
(3, 0)
(3, 1)
(3, 2)
(3, 3)
(3, 4)
(3, 5)
(3, 6)
(3, 7)
(3, 8)


In [31]:
import pickle
rfile = open('/Users/barrybrent/run14jun21no13a.txt','r') # xJ polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)


for k in [0..3]: #the polynomials are unchanged, so the extra datum (at k = 2n+2) wasn't necessary
    index=s[k][0]
    poly = s[k][1]
    print (index,poly)
    print "========================================================================="

(-1, 1)
(0, 3/8*x^2 + 1/2)
(1, 69/1024*x^4 - 1/128*x^2 - 3/64)
(2, 1/128*x^6 - 29/864*x^4 + 1/216*x^2 + 1/54)


In [29]:
import pickle
rfile = open('/Users/barrybrent/run1jun21no1.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)

polydata=[]

wfile = open('/Users/barrybrent/run14jun21no14.txt','a') # xJ polynomials
for n in [0..200]:
    data=[]
    for k in [0..2*n+1]:
        print (n,k)
        m = s[k][0]
        poly=x*s[k][1] 
        cf=polynomialCoefficient(n,poly)
        data=data+[[m,cf*m^(2*n)]]
    rl=R.lagrange_polynomial(data)
    polydata=polydata+[[n-1,rl]] 

    
t = pickle.dumps(str(polydata))
wfile.write(t)
wfile.close()

(0, 0)
(0, 1)
(1, 0)
(1, 1)
(1, 2)
(1, 3)
(2, 0)
(2, 1)
(2, 2)
(2, 3)
(2, 4)
(2, 5)
(3, 0)
(3, 1)
(3, 2)
(3, 3)
(3, 4)
(3, 5)
(3, 6)
(3, 7)
(4, 0)
(4, 1)
(4, 2)
(4, 3)
(4, 4)
(4, 5)
(4, 6)
(4, 7)
(4, 8)
(4, 9)
(5, 0)
(5, 1)
(5, 2)
(5, 3)
(5, 4)
(5, 5)
(5, 6)
(5, 7)
(5, 8)
(5, 9)
(5, 10)
(5, 11)
(6, 0)
(6, 1)
(6, 2)
(6, 3)
(6, 4)
(6, 5)
(6, 6)
(6, 7)
(6, 8)
(6, 9)
(6, 10)
(6, 11)
(6, 12)
(6, 13)
(7, 0)
(7, 1)
(7, 2)
(7, 3)
(7, 4)
(7, 5)
(7, 6)
(7, 7)
(7, 8)
(7, 9)
(7, 10)
(7, 11)
(7, 12)
(7, 13)
(7, 14)
(7, 15)
(8, 0)
(8, 1)
(8, 2)
(8, 3)
(8, 4)
(8, 5)
(8, 6)
(8, 7)
(8, 8)
(8, 9)
(8, 10)
(8, 11)
(8, 12)
(8, 13)
(8, 14)
(8, 15)
(8, 16)
(8, 17)
(9, 0)
(9, 1)
(9, 2)
(9, 3)
(9, 4)
(9, 5)
(9, 6)
(9, 7)
(9, 8)
(9, 9)
(9, 10)
(9, 11)
(9, 12)
(9, 13)
(9, 14)
(9, 15)
(9, 16)
(9, 17)
(9, 18)
(9, 19)
(10, 0)
(10, 1)
(10, 2)
(10, 3)
(10, 4)
(10, 5)
(10, 6)
(10, 7)
(10, 8)
(10, 9)
(10, 10)
(10, 11)
(10, 12)
(10, 13)
(10, 14)
(10, 15)
(10, 16)
(10, 17)
(10, 18)
(10, 19)
(10, 20)
(10, 21)
(11, 0)
(11,

(30, 30)
(30, 31)
(30, 32)
(30, 33)
(30, 34)
(30, 35)
(30, 36)
(30, 37)
(30, 38)
(30, 39)
(30, 40)
(30, 41)
(30, 42)
(30, 43)
(30, 44)
(30, 45)
(30, 46)
(30, 47)
(30, 48)
(30, 49)
(30, 50)
(30, 51)
(30, 52)
(30, 53)
(30, 54)
(30, 55)
(30, 56)
(30, 57)
(30, 58)
(30, 59)
(30, 60)
(30, 61)
(31, 0)
(31, 1)
(31, 2)
(31, 3)
(31, 4)
(31, 5)
(31, 6)
(31, 7)
(31, 8)
(31, 9)
(31, 10)
(31, 11)
(31, 12)
(31, 13)
(31, 14)
(31, 15)
(31, 16)
(31, 17)
(31, 18)
(31, 19)
(31, 20)
(31, 21)
(31, 22)
(31, 23)
(31, 24)
(31, 25)
(31, 26)
(31, 27)
(31, 28)
(31, 29)
(31, 30)
(31, 31)
(31, 32)
(31, 33)
(31, 34)
(31, 35)
(31, 36)
(31, 37)
(31, 38)
(31, 39)
(31, 40)
(31, 41)
(31, 42)
(31, 43)
(31, 44)
(31, 45)
(31, 46)
(31, 47)
(31, 48)
(31, 49)
(31, 50)
(31, 51)
(31, 52)
(31, 53)
(31, 54)
(31, 55)
(31, 56)
(31, 57)
(31, 58)
(31, 59)
(31, 60)
(31, 61)
(31, 62)
(31, 63)
(32, 0)
(32, 1)
(32, 2)
(32, 3)
(32, 4)
(32, 5)
(32, 6)
(32, 7)
(32, 8)
(32, 9)
(32, 10)
(32, 11)
(32, 12)
(32, 13)
(32, 14)
(32, 15)
(32, 16)
(32

(42, 81)
(42, 82)
(42, 83)
(42, 84)
(42, 85)
(43, 0)
(43, 1)
(43, 2)
(43, 3)
(43, 4)
(43, 5)
(43, 6)
(43, 7)
(43, 8)
(43, 9)
(43, 10)
(43, 11)
(43, 12)
(43, 13)
(43, 14)
(43, 15)
(43, 16)
(43, 17)
(43, 18)
(43, 19)
(43, 20)
(43, 21)
(43, 22)
(43, 23)
(43, 24)
(43, 25)
(43, 26)
(43, 27)
(43, 28)
(43, 29)
(43, 30)
(43, 31)
(43, 32)
(43, 33)
(43, 34)
(43, 35)
(43, 36)
(43, 37)
(43, 38)
(43, 39)
(43, 40)
(43, 41)
(43, 42)
(43, 43)
(43, 44)
(43, 45)
(43, 46)
(43, 47)
(43, 48)
(43, 49)
(43, 50)
(43, 51)
(43, 52)
(43, 53)
(43, 54)
(43, 55)
(43, 56)
(43, 57)
(43, 58)
(43, 59)
(43, 60)
(43, 61)
(43, 62)
(43, 63)
(43, 64)
(43, 65)
(43, 66)
(43, 67)
(43, 68)
(43, 69)
(43, 70)
(43, 71)
(43, 72)
(43, 73)
(43, 74)
(43, 75)
(43, 76)
(43, 77)
(43, 78)
(43, 79)
(43, 80)
(43, 81)
(43, 82)
(43, 83)
(43, 84)
(43, 85)
(43, 86)
(43, 87)
(44, 0)
(44, 1)
(44, 2)
(44, 3)
(44, 4)
(44, 5)
(44, 6)
(44, 7)
(44, 8)
(44, 9)
(44, 10)
(44, 11)
(44, 12)
(44, 13)
(44, 14)
(44, 15)
(44, 16)
(44, 17)
(44, 18)
(44, 19)
(44

(52, 54)
(52, 55)
(52, 56)
(52, 57)
(52, 58)
(52, 59)
(52, 60)
(52, 61)
(52, 62)
(52, 63)
(52, 64)
(52, 65)
(52, 66)
(52, 67)
(52, 68)
(52, 69)
(52, 70)
(52, 71)
(52, 72)
(52, 73)
(52, 74)
(52, 75)
(52, 76)
(52, 77)
(52, 78)
(52, 79)
(52, 80)
(52, 81)
(52, 82)
(52, 83)
(52, 84)
(52, 85)
(52, 86)
(52, 87)
(52, 88)
(52, 89)
(52, 90)
(52, 91)
(52, 92)
(52, 93)
(52, 94)
(52, 95)
(52, 96)
(52, 97)
(52, 98)
(52, 99)
(52, 100)
(52, 101)
(52, 102)
(52, 103)
(52, 104)
(52, 105)
(53, 0)
(53, 1)
(53, 2)
(53, 3)
(53, 4)
(53, 5)
(53, 6)
(53, 7)
(53, 8)
(53, 9)
(53, 10)
(53, 11)
(53, 12)
(53, 13)
(53, 14)
(53, 15)
(53, 16)
(53, 17)
(53, 18)
(53, 19)
(53, 20)
(53, 21)
(53, 22)
(53, 23)
(53, 24)
(53, 25)
(53, 26)
(53, 27)
(53, 28)
(53, 29)
(53, 30)
(53, 31)
(53, 32)
(53, 33)
(53, 34)
(53, 35)
(53, 36)
(53, 37)
(53, 38)
(53, 39)
(53, 40)
(53, 41)
(53, 42)
(53, 43)
(53, 44)
(53, 45)
(53, 46)
(53, 47)
(53, 48)
(53, 49)
(53, 50)
(53, 51)
(53, 52)
(53, 53)
(53, 54)
(53, 55)
(53, 56)
(53, 57)
(53, 58)
(53, 

(60, 62)
(60, 63)
(60, 64)
(60, 65)
(60, 66)
(60, 67)
(60, 68)
(60, 69)
(60, 70)
(60, 71)
(60, 72)
(60, 73)
(60, 74)
(60, 75)
(60, 76)
(60, 77)
(60, 78)
(60, 79)
(60, 80)
(60, 81)
(60, 82)
(60, 83)
(60, 84)
(60, 85)
(60, 86)
(60, 87)
(60, 88)
(60, 89)
(60, 90)
(60, 91)
(60, 92)
(60, 93)
(60, 94)
(60, 95)
(60, 96)
(60, 97)
(60, 98)
(60, 99)
(60, 100)
(60, 101)
(60, 102)
(60, 103)
(60, 104)
(60, 105)
(60, 106)
(60, 107)
(60, 108)
(60, 109)
(60, 110)
(60, 111)
(60, 112)
(60, 113)
(60, 114)
(60, 115)
(60, 116)
(60, 117)
(60, 118)
(60, 119)
(60, 120)
(60, 121)
(61, 0)
(61, 1)
(61, 2)
(61, 3)
(61, 4)
(61, 5)
(61, 6)
(61, 7)
(61, 8)
(61, 9)
(61, 10)
(61, 11)
(61, 12)
(61, 13)
(61, 14)
(61, 15)
(61, 16)
(61, 17)
(61, 18)
(61, 19)
(61, 20)
(61, 21)
(61, 22)
(61, 23)
(61, 24)
(61, 25)
(61, 26)
(61, 27)
(61, 28)
(61, 29)
(61, 30)
(61, 31)
(61, 32)
(61, 33)
(61, 34)
(61, 35)
(61, 36)
(61, 37)
(61, 38)
(61, 39)
(61, 40)
(61, 41)
(61, 42)
(61, 43)
(61, 44)
(61, 45)
(61, 46)
(61, 47)
(61, 48)
(61, 49

...

(190, 87)
(190, 88)
(190, 89)
(190, 90)
(190, 91)
(190, 92)
(190, 93)
(190, 94)
(190, 95)
(190, 96)
(190, 97)
(190, 98)
(190, 99)
(190, 100)
(190, 101)
(190, 102)
(190, 103)
(190, 104)
(190, 105)
(190, 106)
(190, 107)
(190, 108)
(190, 109)
(190, 110)
(190, 111)
(190, 112)
(190, 113)
(190, 114)
(190, 115)
(190, 116)
(190, 117)
(190, 118)
(190, 119)
(190, 120)
(190, 121)
(190, 122)
(190, 123)
(190, 124)
(190, 125)
(190, 126)
(190, 127)
(190, 128)
(190, 129)
(190, 130)
(190, 131)
(190, 132)
(190, 133)
(190, 134)
(190, 135)
(190, 136)
(190, 137)
(190, 138)
(190, 139)
(190, 140)
(190, 141)
(190, 142)
(190, 143)
(190, 144)
(190, 145)
(190, 146)
(190, 147)
(190, 148)
(190, 149)
(190, 150)
(190, 151)
(190, 152)
(190, 153)
(190, 154)
(190, 155)
(190, 156)
(190, 157)
(190, 158)
(190, 159)
(190, 160)
(190, 161)
(190, 162)
(190, 163)
(190, 164)
(190, 165)
(190, 166)
(190, 167)
(190, 168)
(190, 169)
(190, 170)
(190, 171)
(190, 172)
(190, 173)
(190, 174)
(190, 175)
(190, 176)
(190, 177)
(190, 178)
(

(192, 98)
(192, 99)
(192, 100)
(192, 101)
(192, 102)
(192, 103)
(192, 104)
(192, 105)
(192, 106)
(192, 107)
(192, 108)
(192, 109)
(192, 110)
(192, 111)
(192, 112)
(192, 113)
(192, 114)
(192, 115)
(192, 116)
(192, 117)
(192, 118)
(192, 119)
(192, 120)
(192, 121)
(192, 122)
(192, 123)
(192, 124)
(192, 125)
(192, 126)
(192, 127)
(192, 128)
(192, 129)
(192, 130)
(192, 131)
(192, 132)
(192, 133)
(192, 134)
(192, 135)
(192, 136)
(192, 137)
(192, 138)
(192, 139)
(192, 140)
(192, 141)
(192, 142)
(192, 143)
(192, 144)
(192, 145)
(192, 146)
(192, 147)
(192, 148)
(192, 149)
(192, 150)
(192, 151)
(192, 152)
(192, 153)
(192, 154)
(192, 155)
(192, 156)
(192, 157)
(192, 158)
(192, 159)
(192, 160)
(192, 161)
(192, 162)
(192, 163)
(192, 164)
(192, 165)
(192, 166)
(192, 167)
(192, 168)
(192, 169)
(192, 170)
(192, 171)
(192, 172)
(192, 173)
(192, 174)
(192, 175)
(192, 176)
(192, 177)
(192, 178)
(192, 179)
(192, 180)
(192, 181)
(192, 182)
(192, 183)
(192, 184)
(192, 185)
(192, 186)
(192, 187)
(192, 188)
(

(194, 104)
(194, 105)
(194, 106)
(194, 107)
(194, 108)
(194, 109)
(194, 110)
(194, 111)
(194, 112)
(194, 113)
(194, 114)
(194, 115)
(194, 116)
(194, 117)
(194, 118)
(194, 119)
(194, 120)
(194, 121)
(194, 122)
(194, 123)
(194, 124)
(194, 125)
(194, 126)
(194, 127)
(194, 128)
(194, 129)
(194, 130)
(194, 131)
(194, 132)
(194, 133)
(194, 134)
(194, 135)
(194, 136)
(194, 137)
(194, 138)
(194, 139)
(194, 140)
(194, 141)
(194, 142)
(194, 143)
(194, 144)
(194, 145)
(194, 146)
(194, 147)
(194, 148)
(194, 149)
(194, 150)
(194, 151)
(194, 152)
(194, 153)
(194, 154)
(194, 155)
(194, 156)
(194, 157)
(194, 158)
(194, 159)
(194, 160)
(194, 161)
(194, 162)
(194, 163)
(194, 164)
(194, 165)
(194, 166)
(194, 167)
(194, 168)
(194, 169)
(194, 170)
(194, 171)
(194, 172)
(194, 173)
(194, 174)
(194, 175)
(194, 176)
(194, 177)
(194, 178)
(194, 179)
(194, 180)
(194, 181)
(194, 182)
(194, 183)
(194, 184)
(194, 185)
(194, 186)
(194, 187)
(194, 188)
(194, 189)
(194, 190)
(194, 191)
(194, 192)
(194, 193)
(194, 194)

(196, 86)
(196, 87)
(196, 88)
(196, 89)
(196, 90)
(196, 91)
(196, 92)
(196, 93)
(196, 94)
(196, 95)
(196, 96)
(196, 97)
(196, 98)
(196, 99)
(196, 100)
(196, 101)
(196, 102)
(196, 103)
(196, 104)
(196, 105)
(196, 106)
(196, 107)
(196, 108)
(196, 109)
(196, 110)
(196, 111)
(196, 112)
(196, 113)
(196, 114)
(196, 115)
(196, 116)
(196, 117)
(196, 118)
(196, 119)
(196, 120)
(196, 121)
(196, 122)
(196, 123)
(196, 124)
(196, 125)
(196, 126)
(196, 127)
(196, 128)
(196, 129)
(196, 130)
(196, 131)
(196, 132)
(196, 133)
(196, 134)
(196, 135)
(196, 136)
(196, 137)
(196, 138)
(196, 139)
(196, 140)
(196, 141)
(196, 142)
(196, 143)
(196, 144)
(196, 145)
(196, 146)
(196, 147)
(196, 148)
(196, 149)
(196, 150)
(196, 151)
(196, 152)
(196, 153)
(196, 154)
(196, 155)
(196, 156)
(196, 157)
(196, 158)
(196, 159)
(196, 160)
(196, 161)
(196, 162)
(196, 163)
(196, 164)
(196, 165)
(196, 166)
(196, 167)
(196, 168)
(196, 169)
(196, 170)
(196, 171)
(196, 172)
(196, 173)
(196, 174)
(196, 175)
(196, 176)
(196, 177)
(1

(198, 67)
(198, 68)
(198, 69)
(198, 70)
(198, 71)
(198, 72)
(198, 73)
(198, 74)
(198, 75)
(198, 76)
(198, 77)
(198, 78)
(198, 79)
(198, 80)
(198, 81)
(198, 82)
(198, 83)
(198, 84)
(198, 85)
(198, 86)
(198, 87)
(198, 88)
(198, 89)
(198, 90)
(198, 91)
(198, 92)
(198, 93)
(198, 94)
(198, 95)
(198, 96)
(198, 97)
(198, 98)
(198, 99)
(198, 100)
(198, 101)
(198, 102)
(198, 103)
(198, 104)
(198, 105)
(198, 106)
(198, 107)
(198, 108)
(198, 109)
(198, 110)
(198, 111)
(198, 112)
(198, 113)
(198, 114)
(198, 115)
(198, 116)
(198, 117)
(198, 118)
(198, 119)
(198, 120)
(198, 121)
(198, 122)
(198, 123)
(198, 124)
(198, 125)
(198, 126)
(198, 127)
(198, 128)
(198, 129)
(198, 130)
(198, 131)
(198, 132)
(198, 133)
(198, 134)
(198, 135)
(198, 136)
(198, 137)
(198, 138)
(198, 139)
(198, 140)
(198, 141)
(198, 142)
(198, 143)
(198, 144)
(198, 145)
(198, 146)
(198, 147)
(198, 148)
(198, 149)
(198, 150)
(198, 151)
(198, 152)
(198, 153)
(198, 154)
(198, 155)
(198, 156)
(198, 157)
(198, 158)
(198, 159)
(198, 160)

In [5]:
import pickle
rfile = open('/Users/barrybrent/run14jun21no14.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
len(s)

201

In [9]:
import pickle
rfile = open('/Users/barrybrent/run14jun21no14.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)

wfile = open('/Users/barrybrent/run14jun21no20.txt','a') # xJ polynomials
data=[]
for k in [0..100]:
    print k
    data=data+[s[k]]
t = pickle.dumps(str(data))
wfile.write(t)
wfile.close()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100


In [19]:
import pickle
rfile = open('/Users/barrybrent/run14jun21no14.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)

wfile = open('/Users/barrybrent/run14jun21no32.txt','a') # xJ polynomials
data=[]
for k in [101..120]:
    print k
    data=data+[s[k]]
t = pickle.dumps(str(data))
wfile.write(t)
wfile.close()

101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120


In [2]:
import pickle
rfile = open('/Users/barrybrent/run14jun21no14.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)

wfile = open('/Users/barrybrent/run14jun21no37.txt','a')               
             
data=[]
for k in [121..135]:
    print k
    data=data+[s[k]]
t = pickle.dumps(str(data))
wfile.write(t)
wfile.close()

121
122
123
124
125
126
127
128
129
130
131
132
133
134
135


In [3]:
import pickle
rfile = open('/Users/barrybrent/run14jun21no14.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)

wfile = open('/Users/barrybrent/run14jun21no38.txt','a')               
             
data=[]
for k in [136..145]:
    print k
    data=data+[s[k]]
t = pickle.dumps(str(data))
wfile.write(t)
wfile.close()

136
137
138
139
140
141
142
143
144
145


In [4]:
import pickle
rfile = open('/Users/barrybrent/run14jun21no14.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)

wfile = open('/Users/barrybrent/run14jun21no39.txt','a')               
             
data=[]
for k in [146..155]:
    print k
    data=data+[s[k]]
t = pickle.dumps(str(data))
wfile.write(t)
wfile.close()

146
147
148
149
150
151
152
153
154
155


In [5]:
import pickle
rfile = open('/Users/barrybrent/run14jun21no14.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)

wfile = open('/Users/barrybrent/run14jun21no60.txt','a')               
             
data=[]
for k in [156..164]:
    print k
    data=data+[s[k]]
t = pickle.dumps(str(data))
wfile.write(t)
wfile.close()

156
157
158
159
160
161
162
163
164


In [6]:
import pickle
rfile = open('/Users/barrybrent/run14jun21no14.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)

wfile = open('/Users/barrybrent/run14jun21no61.txt','a')               
             
data=[]
for k in [165..172]:
    print k
    data=data+[s[k]]
t = pickle.dumps(str(data))
wfile.write(t)
wfile.close()

165
166
167
168
169
170
171
172


In [7]:
import pickle
rfile = open('/Users/barrybrent/run14jun21no14.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)

wfile = open('/Users/barrybrent/run14jun21no62.txt','a')               
             
data=[]
for k in [173..179]:
    print k
    data=data+[s[k]]
t = pickle.dumps(str(data))
wfile.write(t)
wfile.close()

173
174
175
176
177
178
179


In [8]:
import pickle
rfile = open('/Users/barrybrent/run14jun21no14.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)

wfile = open('/Users/barrybrent/run14jun21no63.txt','a')               
             
data=[]
for k in [180..187]:
    print k
    data=data+[s[k]]
t = pickle.dumps(str(data))
wfile.write(t)
wfile.close()

180
181
182
183
184
185
186
187


In [11]:
import pickle
rfile = open('/Users/barrybrent/run14jun21no14.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)

wfile = open('/Users/barrybrent/run14jun21no66.txt','a')               
             
data=[]
for k in [188..192]:
    print k
    data=data+[s[k]]
t = pickle.dumps(str(data))
wfile.write(t)
wfile.close()

188
189
190
191
192


In [13]:
import pickle
rfile = open('/Users/barrybrent/run14jun21no14.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)

wfile = open('/Users/barrybrent/run14jun21no68.txt','a')               
             
data=[]
for k in [193..197]:
    print k
    data=data+[s[k]]
t = pickle.dumps(str(data))
wfile.write(t)
wfile.close()

193
194
195
196
197


In [14]:
import pickle
rfile = open('/Users/barrybrent/run14jun21no14.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)

wfile = open('/Users/barrybrent/run14jun21no69.txt','a')               
             
data=[]
for k in [198..200]:
    print k
    data=data+[s[k]]
t = pickle.dumps(str(data))
wfile.write(t)
wfile.close()

198
199
200
